In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import pandas as pd
import duck_db_helper
import duckdb

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/analytical_sandboxes/analytical_sandbox_zone.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()
dfm = df_list[0]
dfp = df_list[1]

# Joining the two tables

We will add for each player record of a match, the match statistics we have from the matches tables. This will add new features like the betting odds and the weather. 


In [4]:
df_home = pd.merge(dfp, dfm, left_on=['team_x', 'match_date'], right_on=['HomeTeam', 'Date'], how='inner')
df_away = pd.merge(dfp, dfm, left_on=['team_x', 'match_date'], right_on=['AwayTeam', 'Date'], how='inner')
df = pd.concat([df_home, df_away], ignore_index=True)
df

,match_date,team_x,opp_team_name,was_home,name,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,value,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AvgH,AvgD,AvgA,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW
0,2022-08-14,Nott'm Forest,West Ham,1,Brennan Johnson,FWD,0,0,5,1,14.3,394,0,0,3.3,1.6,90,0,0,0,1,0,2,0,160914,17,1,60,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
1,2022-08-14,Nott'm Forest,West Ham,1,Orel Mangala,MID,0,0,4,1,4.4,536,0,0,1.4,1.6,83,0,0,0,0,0,2,0,3453,8,3,50,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
2,2022-08-14,Nott'm Forest,West Ham,1,Dean Henderson,GK,0,3,43,1,0.0,398,0,0,5.0,50.0,90,0,0,1,1,0,2,4,466426,0,14,45,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
3,2022-08-14,Nott'm Forest,West Ham,1,Taiwo Awoniyi,FWD,0,2,26,1,7.2,397,0,1,5.6,31.6,71,0,0,0,0,0,2,0,78654,17,8,59,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
4,2022-08-14,Nott'm Forest,West Ham,1,Harry Toffolo,DEF,0,1,24,1,26.8,523,0,0,4.7,20.6,90,0,0,0,0,0,2,0,8329,0,7,45,2022-08-14,Nott'm Forest,West Ham,1,0,H,4.21,3.64,1.90,1012.9,NE,5.0,0.0,17.533333,13.566667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7300,2023-05-28,Wolves,Arsenal,0,Matheus Luiz Nunes,MID,0,0,3,0,0.0,589,3,0,0.0,0.0,45,0,0,0,0,0,38,0,17059,0,1,48,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7301,2023-05-28,Wolves,Arsenal,0,Rúben da Silva Neves,MID,0,0,4,0,0.0,480,2,0,0.0,0.0,45,0,0,0,0,0,38,0,175772,0,1,52,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7302,2023-05-28,Wolves,Arsenal,0,Nélson Cabral Semedo,DEF,0,0,15,0,0.0,482,5,0,0.0,0.0,90,0,0,0,0,0,38,0,73746,0,0,50,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000
7303,2023-05-28,Wolves,Arsenal,0,Nathan Collins,DEF,0,0,14,0,0.0,516,5,0,0.0,0.0,90,0,0,0,0,0,38,0,57150,0,0,42,2023-05-28,Arsenal,Wolves,5,0,H,1.37,5.33,7.87,1023.4,NE,3.0,0.0,12.100000,8.100000


Now we will get rid of some of the columns used for the merge that are no longer needed

In [5]:
cols_to_remove = ['Date','HomeTeam','AwayTeam']
cols_to_remove = [col for col in cols_to_remove if col in df.columns]
df = df.drop(cols_to_remove,axis=1)

# Generating odds for home or not

We do simple feature generation to get the odds for the players team and the odds agaings the players team by looking at if the player was at home or not.
TODO if this is the only feature generation we are going to do, i suggest we remove all the game from the analytical sandbox. They just clutter up the tables and we never use them.

In [6]:
home_odds_col = 'AvgH'
draw_odds_col = 'AvgD'
away_odds_col = 'AvgA'

df['odds_against'] = 0.0
df['odds_for'] = 0.0

# Fill in odds columns based on whether the player was home or not
df.loc[df['was_home'] == 1, 'odds_for'] = df.loc[df['was_home'] == 1, home_odds_col]
df.loc[df['was_home'] == 0, 'odds_for'] = df.loc[df['was_home'] == 0, away_odds_col]
df.loc[df['was_home'] == 1, 'odds_against'] = df.loc[df['was_home'] == 1, away_odds_col]
df.loc[df['was_home'] == 0, 'odds_against'] = df.loc[df['was_home'] == 0, home_odds_col]

df.drop(columns=[home_odds_col, away_odds_col], inplace=True)
df.rename(columns={'AvgD': 'odds_draw'}, inplace=True)

In [7]:
df.loc[df.was_home==0].head(5)

,match_date,team_x,opp_team_name,was_home,name,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,value,FTHG,FTAG,FTR,odds_draw,PRESS,WDIR,WSPD,CLOUD,TEMP,TDEW,odds_against,odds_for
3658,2022-08-13,Newcastle,Brighton,0,Fabian Schär,DEF,0,0,22,1,0.7,366,0,0,2.1,20.6,90,0,0,0,1,0,2,0,411488,0,5,46,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3659,2022-08-13,Newcastle,Brighton,0,Joe Willock,MID,0,0,11,1,5.2,373,0,0,5.2,20.2,75,0,0,0,0,0,2,0,55473,27,3,50,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3660,2022-08-13,Newcastle,Brighton,0,Kieran Trippier,DEF,0,1,28,1,15.9,357,0,0,3.7,20.6,90,0,0,0,0,0,2,0,1867358,0,7,50,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3661,2022-08-13,Newcastle,Brighton,0,Joelinton Cássio Apolinário de Lira,MID,0,0,10,1,14.0,371,0,0,3.0,14.2,90,0,0,0,1,0,2,0,77813,2,2,60,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1
3662,2022-08-13,Newcastle,Brighton,0,Sven Botman,DEF,0,1,28,1,0.3,377,0,0,1.2,11.8,90,0,0,0,0,0,2,0,186217,0,7,45,0,0,D,3.25,1015.3,NE,4.0,0.0,21.2,13.9,2.44,3.1


# Generate goals and goals against of the team and results according to the team

In [8]:
# Calculate total_goals and total_against based on home or away status
df['team_goals'] = df.apply(lambda x: x['FTHG'] if x['was_home'] == 1 else x['FTAG'], axis=1)
df['team_goals_against'] = df.apply(lambda x: x['FTAG'] if x['was_home'] == 1 else x['FTHG'], axis=1)

# Determine match outcome (win, lose, draw)
def match_outcome(row):
    if (row['was_home'] == 1 and row['FTR'] == 'H') or (row['was_home'] == 0 and row['FTR'] == 'A'):
        return 'win'
    elif (row['was_home'] == 1 and row['FTR'] == 'A') or (row['was_home'] == 0 and row['FTR'] == 'H'):
        return 'lose'
    else:
        return 'draw'

df['match_outcome'] = df.apply(match_outcome, axis=1)

df.drop(columns=['FTHG', 'FTAG', 'FTR'], inplace=True)

# Generate if the player is on a hot streak or cold streak

In [9]:
df = df.sort_values(by=['name', 'match_date'])

def calculate_streak(player_group):
    streaks = []
    win_count = 0
    
    for outcome in player_group['match_outcome']:
        if outcome == 'win':
            win_count += 1
        else:
            win_count = 0
        
        # If the last three matches are wins, streak is 'hot', else 'cold'
        if win_count > 3:
            streaks.append('hot')
        else:
            streaks.append('cold')

    player_group['streak'] = streaks
    return player_group

df = df.groupby('name', group_keys=False).apply(calculate_streak)
df = df.sort_values(by=['match_date','team_x','name'])

# Generate average points in the last 3 matches

In [10]:
def calculate_avg_points_last_3_matches(player_group):
    last_points = []
    average_last_3 = []

    for points in player_group['total_points']:
        average_last_3.append(round(sum(last_points)/len(last_points)) if last_points else 0)

        last_points.append(points)
        if len(last_points) > 3:
            last_points.pop(0)

    player_group['last_3_matches_average'] = average_last_3
    return player_group

df = df.sort_values(by=['name', 'match_date'])

df = df.groupby('name', group_keys=False).apply(calculate_avg_points_last_3_matches)

df = df.sort_values(by=['match_date', 'team_x', 'name'])

# Renaming and reordering columns

In [12]:
df.rename(columns={
    'team_x': 'team', 
    'opp_team_name': 'opponent', 
    'PRESS': 'pressure',
    'WDIR': 'wind_direction',
    'WSPD': 'wind_speed',
    'CLOUD': 'cloud_coverage',
    'TEMP': 'temperature',
    'TDEW': 'dew_point'
}, inplace=True)

In [13]:
new_order = ['name','match_date', 'team', 'opponent', 'was_home', 'team_goals', 'team_goals_against',
             'match_outcome', 'streak', 'position', 'assists', 'bonus', 'bps', 'clean_sheets', 'creativity', 
             'element', 'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes', 'own_goals',
             'penalties_missed', 'penalties_saved', 'yellow_cards', 'red_cards', 'round', 'saves', 'selected',
             'threat', 'total_points', 'last_3_matches_average', 'value', 'pressure', 'wind_direction', 'wind_speed', 'cloud_coverage', 
             'temperature', 'dew_point', 'odds_against', 'odds_for', 'odds_draw']

In [14]:
df = df[new_order]

In [15]:
df

,name,match_date,team,opponent,was_home,team_goals,team_goals_against,match_outcome,streak,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,last_3_matches_average,value,pressure,wind_direction,wind_speed,cloud_coverage,temperature,dew_point,odds_against,odds_for,odds_draw
94,Aaron Ramsdale,2022-08-13,Arsenal,Leicester,1,4,2,win,cold,GK,0,0,10,0,0.0,15,2,0,1.2,11.6,90,0,0,0,0,0,2,1,1369348,0,1,0,50,1016.5,NE,5.0,0.0,22.5,10.7,6.04,1.55,4.37
89,Ainsley Maitland-Niles,2022-08-13,Arsenal,Leicester,1,4,2,win,cold,MID,0,0,0,0,0.0,21,0,0,0.0,0.0,0,0,0,0,0,0,2,0,5165,0,0,0,50,1016.5,NE,5.0,0.0,22.5,10.7,6.04,1.55,4.37
97,Benjamin White,2022-08-13,Arsenal,Leicester,1,4,2,win,cold,DEF,0,0,8,0,3.3,10,2,0,1.3,9.8,74,0,0,0,0,0,2,0,567751,0,1,0,45,1016.5,NE,5.0,0.0,22.5,10.7,6.04,1.55,4.37
91,Bukayo Saka,2022-08-13,Arsenal,Leicester,1,4,2,win,cold,MID,0,0,18,0,43.9,13,2,0,8.1,13.8,83,0,0,0,0,0,2,0,1922750,23,2,0,80,1016.5,NE,5.0,0.0,22.5,10.7,6.04,1.55,4.37
101,Eddie Nketiah,2022-08-13,Arsenal,Leicester,1,4,2,win,cold,FWD,0,0,1,0,0.5,11,0,0,2.0,2.2,6,0,0,0,0,0,2,0,57718,17,1,0,69,1016.5,NE,5.0,0.0,22.5,10.7,6.04,1.55,4.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7302,Nélson Cabral Semedo,2023-05-28,Wolves,Arsenal,0,0,5,lose,cold,DEF,0,0,15,0,0.0,482,5,0,0.0,0.0,90,0,0,0,0,0,38,0,73746,0,0,0,50,1023.4,NE,3.0,0.0,12.1,8.1,1.37,7.87,5.33
7295,Rayan Aït-Nouri,2023-05-28,Wolves,Arsenal,0,0,5,lose,cold,DEF,0,0,4,0,0.0,487,2,0,0.0,0.0,45,0,0,0,0,0,38,0,82370,0,0,0,41,1023.4,NE,3.0,0.0,12.1,8.1,1.37,7.87,5.33
7292,Raúl Jiménez,2023-05-28,Wolves,Arsenal,0,0,5,lose,cold,FWD,0,0,1,0,0.0,476,5,0,0.0,0.0,90,0,0,0,0,0,38,0,34301,0,2,0,65,1023.4,NE,3.0,0.0,12.1,8.1,1.37,7.87,5.33
7301,Rúben da Silva Neves,2023-05-28,Wolves,Arsenal,0,0,5,lose,cold,MID,0,0,4,0,0.0,480,2,0,0.0,0.0,45,0,0,0,0,0,38,0,175772,0,1,2,52,1023.4,NE,3.0,0.0,12.1,8.1,1.37,7.87,5.33


In [17]:
df[df['name']== 'Erling Haaland']

,name,match_date,team,opponent,was_home,team_goals,team_goals_against,match_outcome,streak,position,assists,bonus,bps,clean_sheets,creativity,element,goals_conceded,goals_scored,ict_index,influence,minutes,own_goals,penalties_missed,penalties_saved,yellow_cards,red_cards,round,saves,selected,threat,total_points,last_3_matches_average,value,pressure,wind_direction,wind_speed,cloud_coverage,temperature,dew_point,odds_against,odds_for,odds_draw
46,Erling Haaland,2022-08-13,Man City,Bournemouth,1,4,0,win,cold,FWD,1,0,16,1,13.3,318,0,0,5.5,18.2,73,0,0,0,0,0,2,0,5226268,23,5,0,116,1017.8,E,5.0,0.0,20.8,11.9,37.57,1.07,12.72
3850,Erling Haaland,2022-08-21,Man City,Newcastle,0,3,3,draw,cold,FWD,0,0,28,0,11.9,318,3,1,12.5,43.0,90,0,0,0,0,0,3,0,5676608,70,6,5,117,1010.3,SW,5.0,0.0,10.0,9.2,8.67,1.37,5.11
343,Erling Haaland,2022-08-27,Man City,Crystal Palace,1,4,2,win,cold,FWD,0,3,73,0,1.3,318,2,3,19.4,87.2,83,0,0,0,0,0,4,0,5967805,105,17,6,117,1020.2,N,3.0,0.0,14.0,13.1,15.25,1.19,7.56
525,Erling Haaland,2022-08-31,Man City,Nott'm Forest,1,6,0,win,cold,FWD,0,3,81,1,0.8,318,0,3,18.9,97.2,68,0,0,0,0,0,5,0,6793855,91,17,9,118,1027.4,N,5.0,8.0,14.2,10.0,30.99,1.08,12.84
4287,Erling Haaland,2022-09-17,Man City,Wolves,0,3,0,win,cold,FWD,0,0,29,1,3.4,318,0,1,6.3,37.6,90,0,0,0,0,0,8,0,8407273,22,6,13,120,1023.6,W,5.0,6.0,5.8,4.0,9.25,1.34,5.47
4439,Erling Haaland,2022-10-16,Man City,Liverpool,0,0,1,lose,cold,FWD,0,0,-2,0,13.9,318,1,0,15.4,18.2,90,0,0,0,0,0,11,0,8989666,122,2,13,122,997.6,SW,21.0,8.0,12.4,7.7,3.74,1.91,4.06
1072,Erling Haaland,2022-10-22,Man City,Brighton,1,3,1,win,cold,FWD,0,3,55,0,11.7,318,1,2,13.6,66.4,79,0,0,0,0,0,13,0,8983808,58,13,8,122,998.8,S,7.0,8.0,13.7,12.4,14.36,1.21,6.94
4874,Erling Haaland,2022-10-29,Man City,Leicester,0,1,0,win,cold,FWD,0,0,0,0,0.0,318,0,0,0.0,0.0,0,0,0,0,0,0,14,0,8749219,0,0,7,122,1018.3,S,5.0,8.0,10.8,10.8,9.66,1.29,6.09
5077,Erling Haaland,2022-12-28,Man City,Leeds,0,3,1,win,cold,FWD,0,3,51,0,11.2,318,1,2,19.6,79.6,90,0,0,0,0,0,17,0,9219074,105,13,5,122,1002.9,W,12.0,5.0,7.6,4.1,9.46,1.30,5.99
1582,Erling Haaland,2022-12-31,Man City,Everton,1,1,1,draw,cold,FWD,0,1,20,0,1.9,318,1,1,6.2,27.4,90,0,0,0,1,0,18,0,9300515,33,6,9,122,993.8,SSW,15.0,8.0,9.5,5.7,21.51,1.14,8.61


In [16]:
conn = duckdb.connect('../data/feature_generation/feature_generation.db')
duck_db_helper.create_table('feature_1', df, conn)
conn.close()